In [0]:
# importing necessary packages
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType, ArrayType
from pyspark.sql.functions import col
# creating the class(ReadingData) for reading the data from different source

class ReadingData :
    def __init__(self):
        self.spark = SparkSession.builder.getOrCreate()
        self.dataframe = None
        self.exdf = None
        # self.expected_schema = None
    
    def read_storage(self):
        pass
    def read_snowflake(self):
        pass
    def read_dbfs(self):
        file_path = input("Enter the file path: ")
        file_format = file_path.split('.')[-1].lower()

        reader = self.spark.read.format(file_format)

        if file_format == 'csv':
            reader = reader.option('header', 'true').option('inferSchema', 'true')
        elif file_format == 'parquet':
            # No additional options required for Parquet format
            pass
        elif file_format == 'json':
            
            pass
        elif file_format == 'text':
            # Read text files as a whole file with one row per line
            reader = reader.text()
        else:
            raise ValueError(f"Unsupported file format: {file_format}")

        self.dataframe = reader.load(file_path)
        self.dataframe.show()
        return self.dataframe
    

    
    def get_meta_data(self):
        print("======================================================================================")
        print(" DataFrame MetaData ")
        print("--------------------------------------------------------------------------------------")
        print("the schema of the DataFrame, including column names, data types, and nullable flags.")
        print("--------------------------------------------------------------------------------------")
        print(self.dataframe.printSchema())
        print("--------------------------------------------------------------------------------------")
        print(" the StructType object representing the schema of the DataFrame.")
        print("--------------------------------------------------------------------------------------")
        print(self.dataframe.schema)
        print("--------------------------------------------------------------------------------------")
        print(" a list of column names in the DataFrame.")
        print("--------------------------------------------------------------------------------------")
        print(self.dataframe.columns)
        print("--------------------------------------------------------------------------------------")
        print(" a list of (column_name, data_type) tuples for each column in the DataFrame")
        print("--------------------------------------------------------------------------------------")
        print(self.dataframe.dtypes)
        print("======================================================================================")

class schema_validation(ReadingData):
    


    def take_expected_schema(self):
        # taking expected schema from user as input 
        userinput = input('''how you wnat to input the expected schema for validation 
              enter 1.(D) for default 2. (F)for file 3.(T) TO enter through terminal''')

        if userinput.upper() == "D":
              self.expected_schema = StructType([
                StructField("PassengerId", IntegerType(), nullable=True),
                StructField("Survived", IntegerType(), nullable=True),
                StructField("Pclass", IntegerType(), nullable=True),
                StructField("Name", StringType(), nullable=True),
                StructField("Sex", StringType(), nullable=True),
                StructField("Age", DoubleType(), nullable=True),
                StructField("SibSp", IntegerType(), nullable=True),
                StructField("Parch", IntegerType(), nullable=True),
                StructField("Ticket", StringType(), nullable=True),
                StructField("Fare", DoubleType(), nullable=True),
                StructField("Cabin", StringType(), nullable=True),
                StructField("Embarked", StringType(), nullable=True)
            ])
        
        elif userinput.upper() == "F":
            file_path = input("Enter the file path: ")
            file_format = file_path.split('.')[-1].lower()

            reader = self.spark.read.format(file_format)

            if file_format == 'csv':
                reader = reader.option('header', 'true').option('inferSchema', 'true')
            elif file_format == 'parquet':
                # No additional options required for Parquet format
                pass
            elif file_format == 'json':
                
                pass
            elif file_format == 'text':
                # Read text files as a whole file with one row per line
                reader = reader.text()
            else:
                raise ValueError(f"Unsupported file format: {file_format}")
            self.exdf = reader.load(file_path)
            self.expected_schema = self.exdf.schema
        elif userinput.upper() == "T":
            self.expected_schema = StructType()
            num_fields = int(input("Enter the number of fields in the expected schema: "))

            for i in range(num_fields):
                field_name = input(f"Enter the name for field {i + 1}: ")
                field_type = input(f"Enter the data type for field {i + 1}: ")
                nullable = input(f"Is field {i + 1} nullable? (y/n): ").lower()

                if field_type.lower() == "string":
                    data_type = StringType()
                elif field_type.lower() == "integer":
                    data_type = IntegerType()
                elif field_type.lower() == "double":
                    data_type = DoubleType()
                elif field_type.lower() == "array":
                    element_type = input(f"Enter the element data type for the array field {i + 1}: ")
                    if element_type.lower() == "string":
                        data_type = ArrayType(StringType())
                    elif element_type.lower() == "integer":
                        data_type = ArrayType(IntegerType())
                    elif element_type.lower() == "double":
                        data_type = ArrayType(DoubleType())
                    else:
                        print(f"Invalid element data type entered for the array field {i + 1}. Using ArrayType(StringType) as default.")
                        data_type = ArrayType(StringType())
                else:
                    print(f"Invalid data type entered for field {i + 1}. Using StringType as default.")
                    data_type = StringType()

                if nullable == "y":
                    is_nullable = True
                else:
                    is_nullable = False
                
                field = StructField(field_name, data_type, 
                nullable=is_nullable)
                self.expected_schema.add(field)
        else: 
            print("enter among given option")

            # Compare actual DataFrame schema with expected schema
    def validat_schema (self):
            actual_schema = self.dataframe.schema

            if actual_schema == self.expected_schema:
                print("Schema validation passed.")
            else:
                print("Schema validation failed.")
                print("Expected Schema:")
                print (self.expected_schema)
                print("Actual Schema:")
                self.dataframe.printSchema()

class checking_tables():
    def __init__(self,dataframe):
        self.dataframe = dataframe
    

    def check_table(self):
            #  converting dataframe table view name as "my_table"
            self.dataframe.createOrReplaceTempView("my_table")
            
            # Define tables to validate
            tables = ["my_table"]

            # Perform validation for each table
            for table in tables:
                
                    # Read table data
                    df = spark.sql(f"SELECT * FROM {table}")
                    
                    # Perform validation checks
                    row_count = df.count()
                    column_count = len(df.columns)
                    has_null_values = df.rdd.flatMap(lambda row: row).map(lambda x: x is None).sum()
                    

                # Iterate over each column and calculate the percentage of null values
                    for column in df.columns:
                        null_count = df.filter(col(column).isNull()).count()
                        null_percentage = (null_count / row_count) * 100
                        print(f"Column '{column}': {null_percentage:.2f}% null")
                        print("-----------------------------------------------------")
                        
                    
                    # Print validation results
                    print(f"Validation Results for Table: {table}")
                    print(f"Total Rows: {row_count}")
                    print(f"Total Columns: {column_count}")
                    print(f"Null Values: {has_null_values}")
                    
                    print("---\n")
    

In [0]:
getData = schema_validation()

# print("define the source you want to get datafrom")
# filedes = input("enter the 1. (F) for snowflake 2. (D) for dbfs 3. (S) from storage")
# if filedes.upper == 'F':
    # getData.read_snowflake()
# elif filedes.upper == 'D':
dataframe  = getData.read_dbfs()

# elif filedes.upper == 'S':
#     getData.read_storage()
# else:
#     print("please enter through the given source")

getData.get_meta_data()
    
getData.take_expected_schema()
getData.validat_schema()


table_info = checking_tables(dataframe)
table_info.check_table()


# file_path = "/FileStore/tables/titanic.csv"Read any type of file format and convert it into pyspark dataframe 
# file_path ="/FileStore/tables/sample1.json"

Enter the file path:  /FileStore/tables/titanic.csv

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

how you wnat to input the expected schema for validation 
              enter 1.(D) for default 2. (F)for file 3.(T) TO enter through terminal d

Schema validation passed.
Column 'PassengerId': 0.00% null
-----------------------------------------------------
Column 'Survived': 0.00% null
-----------------------------------------------------
Column 'Pclass': 0.00% null
-----------------------------------------------------
Column 'Name': 0.00% null
-----------------------------------------------------
Column 'Sex': 0.00% null
-----------------------------------------------------
Column 'Age': 19.87% null
-----------------------------------------------------
Column 'SibSp': 0.00% null
-----------------------------------------------------
Column 'Parch': 0.00% null
-----------------------------------------------------
Column 'Ticket': 0.00% null
-----------------------------------------------------
Column 'Fare': 0.00% null
-----------------------------------------------------
Column 'Cabin': 77.10% null
-----------------------------------------------------
Column 'Embarked': 0.22% null
----------------------------------------------